# ECS 171 Group Project - Group 10 - Reference ANN

In [5]:
import numpy as np
import pandas as pd

np.random.seed(171)

#Contains 81 features extracted from 21263 superconductors along with the critical temperature in the 82nd column
X_train = pd.read_csv('./../../Data/X_train.csv', header=None)
Y_train = pd.read_csv('./../../Data/Y_train.csv', header=None)

X_test  = pd.read_csv('./../../Data/X_test.csv', header=None)
Y_test  = pd.read_csv('./../../Data/Y_test.csv', header=None)

## Untuned Reference Neural Network

In [55]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense

# source: https://jmlb.github.io/ml/2017/03/20/CoeffDetermination_CustomMetric4Keras/
from tensorflow.keras import backend as K
# calculate R^2
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

nn = Sequential(
    [
        keras.Input(shape=X_train.shape[1]),
        layers.Dense(20, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(20, activation='relu'),
        layers.Dense(1)
    ]
)


nn.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
           loss="mean_squared_error",
           metrics=[coeff_determination])

NUM_EPOCHS = 100
history = nn.fit(X_train, Y_train, batch_size=64, epochs=NUM_EPOCHS, verbose=0)

pred = nn.predict(X_test)
results = nn.evaluate(X_test, Y_test)

print()
print("train loss: " + str(history.history['loss'][NUM_EPOCHS-1]))
print("train R^2:  " + str(history.history['coeff_determination'][NUM_EPOCHS-1]))
print("test loss:  " + str(results[0]))
print("test  R^2:  " + str(results[1]))

220/220 [==============================] - 0s 449us/step - loss: 224.8168 - coeff_determination: 0.7983

train loss: 153.13894653320312
train R^2:  0.86241614818573
test loss:  224.8167724609375
test  R^2:  0.7983034253120422


## Tuned Neural Network

In [61]:
# source: https://www.sicara.ai/blog/hyperparameter-tuning-keras-tuner
#         https://pythonprogramming.net/keras-tuner-optimizing-neural-network-tutorial/
#.        https://www.youtube.com/watch?v=vvC15l4CY1Q

from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = Sequential()
    model.add(keras.Input(shape=X_train.shape[1]))
    model.add(
        Dense(
            units=hp.Int('units', min_value=10, max_value=40, step=10),
            activation='relu'
        )
    )
    model.add(
        Dense(
            units=hp.Int('units', 30, 70, 10),
            activation='relu'
        )
    )
    model.add(
        Dense(
            units=hp.Int('units', 30, 70, 10),
            activation='relu'
        )
    )
    model.add(
        Dense(
            units=hp.Int('units', 80, 100, 10),
            activation='relu'
        )
    )
    model.add(
        Dense(
            units=hp.Int('units', 30, 70, 10),
            activation='relu'
        )
    )
    model.add(
        Dense(
            units=hp.Int('units', 30, 70, 10),
            activation='relu'
        )
    )
    model.add(
        Dense(
            units=hp.Int('units', 10, 40, 10),
            activation='relu'
        )
    )
    model.add(
        Dense(1, activation='linear')
    )

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float(
                'learning_rate',
                min_value=1e-4,
                max_value=1e-2,
                sampling='LOG',
                default=1e-3
            )
        ),
        loss='mean_squared_error',
        metrics=[keras.metrics.MeanSquaredError()]
    )

    return model

SEED=1
MAX_TRIALS=10
EXECUTION_PER_TRIAL=2

tuner = RandomSearch(
    build_model,
    objective='mean_squared_error',
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='random_search',
    project_name='group10'
)

tuner.search(x=X_train,
             y=Y_train,
             epochs=1,
             batch_size=64,
             validation_data=(X_test,Y_test))

Trial 37 Complete [00h 00m 01s]
mean_squared_error: 768.7422790527344

Best mean_squared_error So Far: 466.96710205078125
Total elapsed time: 00h 01m 03s

Search: Running Trial #38

Hyperparameter    |Value             |Best Value So Far 
units             |10                |20                
learning_rate     |0.0034253         |0.0089386         
units1            |10                |20                
units2            |30                |60                
units3            |70                |40                
units4            |90                |100               
units5            |70                |50                
units6            |40                |40                
units7            |20                |30                



KeyboardInterrupt: 

In [57]:
tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                1640      
_________________________________________________________________
dense_1 (Dense)              (None, 60)                1260      
_________________________________________________________________
dense_2 (Dense)              (None, 40)                2440      
_________________________________________________________________
dense_3 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_5 (Dense)              (None, 40)                2040      
_________________________________________________________________
dense_6 (Dense)              (None, 30)                1